In [1]:
import pandas as pd
import urllib.request, json
#import pydotplus as pydotplus
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
import numpy as np

C:\Users\palmar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Funktsioon, millega saada andmed lehelt kätte. Tagastab andmed kujul [{}, {}, ... ,{}] ehk sõnastikke sisaldav massiiv, kus iga sõnastik hoiab infot ühe taotluse kohta


In [2]:
def getAndmed():
    with urllib.request.urlopen("https://toetused.kul.ee/public/applications/fetch?sort=submission_date%7Cdesc&page=1&per_page=20000000&applicant=&organization=&applicationround=&submissionDate=%7B%22start%22:null,%22end%22:null%7D") as f:
        data = json.load(f)
    andmed = data.get("data")
    return andmed

Tabeli tegemine

In [3]:
def tabel_andmetest():
    rawAndmed = getAndmed()
    
    id = []
    application_code = []
    applicationround_title = []
    approved_amount =[]
    cost_statement_submission_date = []
    cost_statement_submission_deadline = []
    managing_organization_name = []
    name = []
    project_name = []
    registration_date = []
    registry_code = []
    requested_amount = []
    status = []
    status_txt = []
    submission_date = []
    domain_code = []
    domain_name =[]
    
    for taotlus in rawAndmed:
        if taotlus.get("status") == "decision_ok":
            taotlus["status"] = 1
        elif taotlus.get("status") == "decision_not_ok":
            taotlus["status"] = 0
        else:
            taotlus["status"] = None
        
    
    for taotlus in rawAndmed:
        id.append(taotlus.get("id"))
        application_code.append(taotlus.get("application_code"))
        applicationround_title.append(taotlus.get("applicationround_title"))
        approved_amount.append(taotlus.get("approved_amount"))
        cost_statement_submission_date.append(taotlus.get("cost_statement_submission_date"))
        cost_statement_submission_deadline.append(taotlus.get("cost_statement_submission_deadline"))
        managing_organization_name.append(taotlus.get("managing_organization_name"))
        name.append(taotlus.get("name"))
        project_name.append(taotlus.get("project_name"))
        registration_date.append(taotlus.get("registration_date"))
        registry_code.append(taotlus.get("registry_code"))
        requested_amount.append(taotlus.get("requested_amount"))
        status.append(taotlus.get("status"))
        status_txt.append(taotlus.get("status_txt"))
        submission_date.append(taotlus.get("submission_date"))
        domain_code.append(taotlus.get("domain_code"))
        domain_name.append(taotlus.get("domain_name"))
    
    andmed = pd.DataFrame(
        {#"ID" : id,
        "application_code" : application_code,
        "applicationround_title" : applicationround_title,
        "approved_amount" : approved_amount,
        "cost_statement_submission_date" : cost_statement_submission_date,
        "cost_statement_submission_deadline" : cost_statement_submission_deadline,
        "managing_organization_name" : managing_organization_name,
        "name" : name,
        "project_name" : project_name,
        "registration_date" : registration_date,
        "registry_code" : registry_code,
        "requested_amount" : requested_amount,
        "status" : status,
        "status_txt" : status_txt,
        "submission_date" : submission_date,
        "domain_code" : domain_code,
        "domain_name" : domain_name
        }
    )
    return andmed

def decisionTree():
    tabel = tabel_andmetest()
    
    feature_cols = ["requested_amount", "domain_name"]
    treenimiseks = tabel[(tabel.status.notnull())]
    
    X = treenimiseks[feature_cols]
    y = treenimiseks["status"]
    
    enc1 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    X.domain_name = domain_int
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=219)
    dtc = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0, presort=False, random_state=None,
            splitter='best')
    
    dtc = dtc.fit(X_train, y_train)
    
    y_pred = dtc.predict(X_test)
    #print(y_test, y_pred)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    import os
    
    os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
    #dot_data = StringIO()
    #export_graphviz(dtc, out_file=dot_data, filled=True, rounded=True, special_characters=True,feature_names = feature_cols,class_names=['0','1'])
    #graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    #graph.write_png('pilt.png')
    #Image(graph.create_png())
    
    return dtc

def randomforest():
    tabel = tabel_andmetest()
    
    feature_cols = ["domain_name", "requested_amount"]
    treenimiseks = tabel[(tabel.status.notnull())]
    
    X = treenimiseks[feature_cols]
    y = treenimiseks["status"]
    
    enc1 = preprocessing.LabelEncoder()
    enc2 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    managing_org_name_encoder = enc2.fit(tabel.managing_organization_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    #managing_organization_name_int = managing_org_name_encoder.transform(X.managing_organization_name)
    X.domain_name = domain_int
    #X.managing_organization_name = managing_organization_name_int
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    #X, y = make_classification(n_samples=2000, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=False)
    dlf = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=None, 
                                 min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                 max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                 min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, 
                                 random_state=318, verbose=0, warm_start=False, class_weight=None)
    dlf = dlf.fit(X_train, y_train)
    
    y_pred = dlf.predict(X_test)
    print("Forest Accuracy:",metrics.accuracy_score(y_test, y_pred))
    return dlf

def knn():
    tabel = tabel_andmetest()
    
    feature_cols = ["domain_name", "requested_amount"]
    treenimiseks = tabel[(tabel.status.notnull())]
    
    X = treenimiseks[feature_cols]
    y = treenimiseks["status"]
    
    enc1 = preprocessing.LabelEncoder()
    enc2 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    managing_org_name_encoder = enc2.fit(tabel.managing_organization_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    #managing_organization_name_int = managing_org_name_encoder.transform(X.managing_organization_name)
    X.domain_name = domain_int
    #X.managing_organization_name = managing_organization_name_int
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    classifier = KNeighborsClassifier(n_neighbors=31)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print("KNN Accuracy:", metrics.accuracy_score(y_test, y_pred))
    return classifier

def arva(arvamiseks):
    feature_cols = ["domain_name", "requested_amount"]
    X = arvamiseks[feature_cols]
    tabel = tabel_andmetest()
    
    enc1 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    X.domain_name = domain_int
    
    
    õpetaja = knn()
    tulemus = õpetaja.predict(X)
    return tulemus

def arvaLihtandmed(kategooria, summa):
    
    andmed = pd.DataFrame({"domain_name" : [kategooria], "requested_amount" : [summa]})
    return arva(andmed)
    
##print(arvaLihtandmed('Audiovisuaal', 50000000000000)[0])


In [4]:
def biggestFunding():
    data = tabel_andmetest();
    most_expensive = data.loc[data['approved_amount'].idxmax()]
    applicationround_title = most_expensive["applicationround_title"]
    approved_amount = most_expensive["approved_amount"]
    managing_organization_name = most_expensive["managing_organization_name"]
    name = most_expensive["name"]
    project_name = most_expensive["project_name"]
    return applicationround_title, approved_amount, managing_organization_name, name, project_name
    


In [5]:
def smallestFunding():
    data = tabel_andmetest();
    data = data.dropna(how = 'all') 
    data.drop(data[data['approved_amount'] <= 0].index, inplace = True) 
    cheapest = data.loc[data['approved_amount'].idxmin()]
    applicationround_title = cheapest["applicationround_title"]
    approved_amount = cheapest["approved_amount"]
    managing_organization_name = cheapest["managing_organization_name"]
    name = cheapest["name"]
    project_name = cheapest["project_name"]
    return applicationround_title, approved_amount, managing_organization_name, name, project_name


In [6]:
#Väike kasutajaliides 

from tkinter import *
from tkinter import ttk
from tkinter.ttk import *

# GUI
window = Tk()
window.title("OPENDATA ESTONIA - Estonia's culture ministry's grant applications")
window.resizable(False, False)
window.geometry("641x300")
tabControll = ttk.Notebook(window)

tab1 = ttk.Frame(tabControll)
tabControll.add(tab1, text="Introduction")

tab2 = ttk.Frame(tabControll)
tabControll.add(tab2, text="Will Your project get funding?")

tab3 = ttk.Frame(tabControll)
tabControll.add(tab3, text="Top projects with smallest funding")

tab4 = ttk.Frame(tabControll)
tabControll.add(tab4, text="Top projects with biggest funding")

#TAB 1
text9Tab1 = ttk.Label(tab1, text="\n") 
text9Tab1.pack()
text1Tab1 = ttk.Label(tab1, text="We are second year Informatics students at Tartu University.") 
text1Tab1.pack()
text5Tab1 = ttk.Label(tab1, text="Our names are Anneli Klamas, Enrih Sinilaid and Anett-Kristin Palmar.\n") 
text5Tab1.pack()
text2Tab1 = ttk.Label(tab1, text="Our project is about the Culture Ministry's grants.")
text2Tab1.pack()
text6Tab1 = ttk.Label(tab1, text="We were interested in if we could use machine learning to guess which kind of applications would get a grant. \n")
text6Tab1.pack()
text3Tab1 = ttk.Label(tab1, text="You can try to add your own mock-application!")
text3Tab1.pack()
text7Tab1 = ttk.Label(tab1, text="Our programme will then guess if your application would get a grant or not.")
text7Tab1.pack()
text8Tab1 = ttk.Label(tab1, text="Our programme also shows past applications with the least and most grants. \n")
text8Tab1.pack()
text4Tab1 = ttk.Label(tab1, text="Have fun! :)")
text4Tab1.pack()

#TAB 2
text1Tab2 = ttk.Label(tab2, text="Choose domain name")
text1Tab2.grid(column=0, row=0)
combo1Tab2 = Combobox(tab2)
data = tabel_andmetest()
domain_names = data["domain_name"].unique()
combo1Tab2['values'] = domain_names
combo1Tab2.current(1)
combo1Tab2.grid(column=1, row=0)

text2Tab2 = ttk.Label(tab2, text="Enter requested amount")
text2Tab2.grid(column=0, row=1)
entry2Tab2 = Entry(tab2)
entry2Tab2.grid(column=1, row=1)
buttonTab2 = Button(tab2, text="Calculate")
buttonTab2.grid(column=0, row=2)
tabControll.pack(expand=1, fill="both")



#saab kasutada kasutaja sisendi joks
#kasutaja_sisend = ttk.Entry(raam)
#kasutaja_sisend.place(x=120, y=130, height=25, width=240)

#TAB 3
text2Tab3 = ttk.Label(tab3, text="\n")
text2Tab3.pack()
text1Tab3 = ttk.Label(tab3, text="The company with the smallest grant's info (application round, amount, organization, name, project name). \n")
text1Tab3.pack()
Lb3 = Listbox(tab3, width=50, height=5)
Lb3.insert(1, smallestFunding()[0])
Lb3.insert(2, smallestFunding()[1])
Lb3.insert(3, smallestFunding()[2])
Lb3.insert(4, smallestFunding()[3])
Lb3.insert(5, smallestFunding()[4])
Lb3.pack()

#TAB 4
text2Tab4 = ttk.Label(tab4, text="\n")
text2Tab4.pack()
text1Tab4 = ttk.Label(tab4, text="The company with the biggest grant's info (application round, amount, organization, name, project name). \n")
text1Tab4.pack()
Lb4 = Listbox(tab4, width=50, height=5)
Lb4.insert(1, biggestFunding()[0])
Lb4.insert(2, biggestFunding()[1])
Lb4.insert(3, biggestFunding()[2])
Lb4.insert(4, biggestFunding()[3])
Lb4.insert(5, biggestFunding()[4])
Lb4.pack()
window.mainloop()
